# SIIM-COVID19: Classification Only PyTorch Starter

This notebook is just a classification only starter. 

I will be making an Object Detection notebook soon!

### Please leave an upvote if you are forking it or you found it helpful : )

Credits to [this](https://www.kaggle.com/yujiariyasu/catch-up-on-positive-samples-plot-submission-csv) for some of the functions I am using here.

In [ ]:
! conda install -c conda-forge gdcm -y;

In [ ]:
import sys
sys.path.append("../input/timmeffnetv2")

import platform
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import cv2
import pydicom
import gdcm
import glob
import gc
from math import ceil
import matplotlib.pyplot as plt
from pydicom.pixel_data_handlers.util import apply_voi_lut
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

import torch
import timm
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.simplefilter('ignore')

In [ ]:
train_image = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
train_study = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")

In [ ]:
TRAIN_DIR = "../input/siim-covid19-detection/train/"
train_study['StudyInstanceUID'] = train_study['id'].apply(lambda x: x.replace('_study', ''))
train = train_image.merge(train_study, on='StudyInstanceUID')

# Make a path folder
paths = []
for instance_id in tqdm(train['StudyInstanceUID']):
    paths.append(glob.glob(os.path.join(TRAIN_DIR, instance_id +"/*/*"))[0])

train['path'] = paths

train = train.drop(['id_x', 'id_y'], axis=1)

train.head()

In [ ]:
class Config:
    train_pcent = 0.85
    model_name = 'tf_efficientnet_b4'
    image_size = (400, 400)
    TRAIN_BS = 32
    VALID_BS = 16
    num_workers = 8
    NB_EPOCHS = 3
    scaler = GradScaler()

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
class EfficientNetModel(nn.Module):
    """
    Model Class for EfficientNet Model
    """
    def __init__(self, num_classes=4, model_name=Config.model_name, pretrained=True):
        super(EfficientNetModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x
    
class NFNetModel(nn.Module):
    """
    Model Class for EfficientNet Model
    """
    def __init__(self, num_classes=4, model_name=Config.model_name, pretrained=True):
        super(NFNetModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        self.model.head.fc = nn.Linear(self.model.head.fc.in_features, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class Trainer:
    def __init__(self, train_dataloader, valid_dataloader, model, optimizer, loss_fn, val_loss_fn, agc=False, device="cuda:0"):
        """
        Constructor for Trainer class
        """
        self.train = train_dataloader
        self.valid = valid_dataloader
        self.optim = optim
        self.loss_fn = loss_fn
        self.val_loss_fn = val_loss_fn
        self.device = device
        self.agc = agc
    
    def train_one_cycle(self):
        """
        Runs one epoch of training, backpropagation and optimization
        """
        model.train()
        train_prog_bar = tqdm(self.train, total=len(self.train))

        all_train_labels = []
        all_train_preds = []
        
        running_loss = 0
        
        for xtrain, ytrain in train_prog_bar:
            xtrain = xtrain.to(self.device).float()
            ytrain = ytrain.to(self.device).float()
            xtrain = xtrain.permute(0, 3, 1, 2)
            
            with autocast():
                # Get predictions
                z = model(xtrain)

                # Training
                train_loss = self.loss_fn(z, ytrain)
                scaler.scale(train_loss).backward()
                
                if self.agc:
                    adaptive_clip_grad(model.parameters(), clip_factor=0.01, eps=1e-3, norm_type=2.0)
                
                scaler.step(self.optim)
                scaler.update()
                self.optim.zero_grad(set_to_none=True)

                # For averaging and reporting later
                running_loss += train_loss

                # Convert the predictions and corresponding labels to right form
                train_predictions = torch.argmax(z, 1).detach().cpu().numpy()
                train_labels = ytrain.detach().cpu().numpy()

                # Append current predictions and current labels to a list
                all_train_labels += [train_predictions]
                all_train_preds += [train_labels]

            # Show the current loss to the progress bar
            train_pbar_desc = f'loss: {train_loss.item():.4f}'
            train_prog_bar.set_description(desc=train_pbar_desc)
        
        # Now average the running loss over all batches and return
        train_running_loss = running_loss / len(self.train)
        print(f"Final Training Loss: {train_running_loss:.4f}")
        
        # Free up memory
        del all_train_labels, all_train_preds, train_predictions, train_labels, xtrain, ytrain, z
        gc.collect()
        torch.cuda.empty_cache()
        
        return train_running_loss

    def valid_one_cycle(self):
        """
        Runs one epoch of prediction
        """        
        model.eval()
        
        valid_prog_bar = tqdm(self.valid, total=len(self.valid))
        
        with torch.no_grad():
            all_valid_labels = []
            all_valid_preds = []
            
            running_loss = 0
            
            for xval, yval in valid_prog_bar:
                xval = xval.to(self.device).float()
                yval = yval.to(self.device).float()
                xval = xval.permute(0, 3, 1, 2)
                
                val_z = model(xval)
                
                val_loss = self.val_loss_fn(val_z, yval)
                
                running_loss += val_loss.item()
                
                val_pred = torch.argmax(val_z, 1).detach().cpu().numpy()
                val_label = yval.detach().cpu().numpy()
                
                all_valid_labels += [val_label]
                all_valid_preds += [val_pred]
            
                # Show the current loss
                valid_pbar_desc = f"loss: {val_loss.item():.4f}"
                valid_prog_bar.set_description(desc=valid_pbar_desc)
            
            # Get the final loss
            final_loss_val = running_loss / len(self.valid)
            
            # Get Validation Accuracy
            all_valid_labels = np.concatenate(all_valid_labels)
            all_valid_preds = np.concatenate(all_valid_preds)
            
            print(f"Final Validation Loss: {final_loss_val:.4f}")
            
            # Free up memory
            del all_valid_labels, all_valid_preds, val_label, val_pred, xval, yval, val_z
            gc.collect()
            torch.cuda.empty_cache()
            
        return (final_loss_val, model)

In [ ]:
class SIIMData(Dataset):
    def __init__(self, df, is_train=True, augments=None, img_size=Config.image_size):
        super().__init__()
        self.df = df.sample(frac=1).reset_index(drop=True)
        self.is_train = is_train
        self.augments = augments
        self.img_size = img_size
        
    def __getitem__(self, idx):
        image_id = self.df['StudyInstanceUID'].values[idx]
        
        image_path = self.df['path'].values[idx]
        image = dicom2array(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(image, Config.image_size)
        
        # Augments must be albumentations
        if self.augments:
            image = self.augments(image=image)['image']
        else:
            image = torch.tensor(image)
        
        if self.is_train:
            label = self.df[self.df['StudyInstanceUID'] == image_id].values.tolist()[0][3:7]
            return image, torch.tensor(label)
        
        return image
    
    def __len__(self):
        return len(self.df)

In [ ]:
# Training Code
if __name__ == '__main__':
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
        DEVICE = torch.device('cuda:0')
    else:
        print("\n[INFO] GPU not found. Using CPU: {}\n".format(platform.processor()))
        DEVICE = torch.device('cpu')
    
    nb_training_samples = int(Config.train_pcent * train.path.shape[0])
    train_data = train[:nb_training_samples]
    valid_data = train[nb_training_samples:]

    print(f"[INFO] Training on {train_data.shape[0]} samples ({int(Config.train_pcent*100)}%) and validation on {valid_data.shape[0]} ({ceil(abs(1 - Config.train_pcent) * 100)}%) samples")
    
    # Make Training and Validation Datasets
    training_set = SIIMData(
        df=train_data
    )

    validation_set = SIIMData(
        df=valid_data
    )

    train_loader = DataLoader(
        training_set,
        batch_size=Config.TRAIN_BS,
        shuffle=True,
        num_workers=8,
        pin_memory=True
    )

    valid_loader = DataLoader(
        validation_set,
        batch_size=Config.VALID_BS,
        shuffle=False,
        num_workers=8
    )
    
    if "efficient" in Config.model_name or "eff" in Config.model_name:        
        model = EfficientNetModel().to(DEVICE)
    
    elif "nfnet" in Config.model_name:
        model = NFNetModel().to(DEVICE)
    
    else:
        raise RuntimeError("Must specify a valid model type to train.")
    
    print(f"Training Model: {Config.model_name}")
    
    optim = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
    loss_fn_train = nn.BCEWithLogitsLoss()
    loss_fn_val = nn.BCEWithLogitsLoss()

    trainer = Trainer(
        train_dataloader=train_loader,
        valid_dataloader=valid_loader,
        model=model,
        optimizer=optim,
        loss_fn=loss_fn_train,
        val_loss_fn=loss_fn_val,
        agc=False,
        device=DEVICE,
    )

    train_losses_eff = []
    valid_losses_eff = []

    scaler = GradScaler()

    for epoch in range(Config.NB_EPOCHS):
        print(f"{'-'*20} EPOCH: {epoch+1}/{Config.NB_EPOCHS} {'-'*20}")

        # Run one training epoch
        current_train_loss = trainer.train_one_cycle()
        train_losses_eff.append(current_train_loss)

        # Run one validation epoch
        current_val_loss, op_model = trainer.valid_one_cycle()
        valid_losses_eff.append(current_val_loss)

        # Empty CUDA cache
        torch.cuda.empty_cache()